In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import time
import pywifi
from pywifi import const
from urllib.request import urlopen
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
class keepInternet(object):
    ''' 通过切换 wifi 来保持互联网可用
    Args:
        deviceIndex: 无线网卡的设备序号，从 0 开始
    '''

    def __init__(self, deviceIndex=0):
        wifi = pywifi.PyWiFi()
        wifi._logger.setLevel(logging.ERROR)
        # 当前的无线网卡
        self.iface = wifi.interfaces()[deviceIndex]

    def test_usability(self, timeout=5):
        ''' 通过访问向日葵官网来看看是否有网
        Args:
            timeout: 等待时间，默认 5秒
        Returns:
            bool: 是否有网
        '''
        url = 'https://sunlogin.oray.com/personal/'
        try:
            urlopen(url, timeout=5)
        except Exception:
            return False
        return True

    def get_profile(self, ssids):
        ''' 查看已连接过的 wifi
        Args:
            ssids: list, 限定连接的 wifi.
                为空时表示不限制，所有wifi都将可能再次连接
        Returns:
            nets: list contained Profile, 返回筛选过的连接过的 wifi Profile
        '''
        nets = list()
        for net in self.iface.network_profiles():
            if not ssids or net.ssid in ssids:
                nets.append(net)

        return nets

    def scan(self, timeout=5):
        ''' 扫描活动的 wifi
        Returns:
            activeWifi: dict, (ssid, signal)
        '''
        logger.info(f'scan wifi, wait {timeout} sec.')
        self.iface.scan()
        time.sleep(timeout)
        scanRes = self.iface.scan_results()
        logger.info('scan completed...')
        activeWifi = dict()
        for profile in scanRes:
            activeWifi[profile.ssid] = profile.signal
        return activeWifi

    @staticmethod
    def connectableLis(nets_profile, activeWifi):
        ''' 可以直接连接的 wifi
        Args:
            nets_profile: list contained Profile, wifi的信息
            activeWifi: dict, (ssid, signal), 活动的 wifi 
        Returns:
            connectable: list contained Profile, 当前可以直接连接wifi的信息
        '''
        connectable = list()
        for profile in nets_profile:
            if profile.ssid in activeWifi:
                profile.signal = activeWifi[profile.ssid]
                connectable.append(profile)
        connectable.sort(key=lambda item: item.signal, reverse=True)
        return connectable

    def connect(self, profile):
        ''' 连接某个 wifi
        Args:
            profile: list contained Profile, wifi的信息
        Returns:
            bool, 是否连接成功
        '''
        logger.info('----disconnect current wifi.')
        while self.iface.status() not in\
                [const.IFACE_DISCONNECTED, const.IFACE_INACTIVE]:
            self.iface.disconnect()
            time.sleep(1)
        logger.info(f'----disconnected, connecting next wifi: "{profile.ssid}".')
        self.iface.connect(profile)
        time.sleep(10)
        return self.iface.status() == const.IFACE_CONNECTED

    # def connectNewWifi(self, ssid, key):
    #     profile = pywifi.Profile()
    #     profile.ssid = 'QZYS'
    #     profile.auth = const.AUTH_ALG_OPEN
    #     profile.akm.append(const.AKM_TYPE_WPA2PSK)
    #     profile.cipher = const.CIPHER_TYPE_CCMP
    #     profile.key = '12345678'
    #     self.iface.disconnect()
    #     time.sleep(1)
    #     assert self.iface.status() in\
    #         [const.IFACE_DISCONNECTED, const.IFACE_INACTIVE]
    #     self.iface.remove_all_network_profiles()
    #     tmp_profile = self.iface.add_network_profile(profile)

    #     self.iface.connect(tmp_profile)
    #     time.sleep(30)
    #     assert self.iface.status() == const.IFACE_CONNECTED


In [ ]:
if __name__ == '__main__':
    ssids = []
    interval = 2

    interface = keepInternet()

    while True:
        try:
            time.sleep(interval)
            if interface.test_usability():
                logger.info('internet is available.')
                continue
            logger.error(
                'internet is unavailable!!!! prepared to change next wifi.')
            connectable = interface.connectableLis(
                interface.get_profile(ssids),
                interface.scan())
            for profile in connectable:
                if interface.connect(profile) and interface.test_usability():
                    logger.info(
                        f'wifi changed to "{profile.ssid}". Internet is available.')
                    continue
        except Exception as e:
            logger.critical(e)
